In [1]:
import pandas as pd
import json
from pandas import json_normalize

df = pd.read_csv('final_data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147519 entries, 0 to 147518
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   pos          147519 non-null  int64 
 1   artist_name  147519 non-null  object
 2   track_uri    147519 non-null  object
 3   artist_uri   147519 non-null  object
 4   track_name   147518 non-null  object
 5   album_uri    147519 non-null  object
 6   duration_ms  147519 non-null  int64 
 7   album_name   147519 non-null  object
 8   Genre        147519 non-null  object
dtypes: int64(2), object(7)
memory usage: 10.1+ MB


In [5]:
with open ('vetting_playlist.json', 'r') as file:
    vet = json.load(file)

<p>Dancability<br>
Year<br>
Genre<br>
Duration</p>